# Logging
There are two main possibilities to apply logging in Julia:
1. load a specific module ([Logging](https://github.com/kmsquire/Logging.jl)), 
2. use the logging available inside the Base package. 

We have a short comparison between the 2.
## Base package
The basic functions are limited to `info`, `warn` and `error`. No `debug` or `critical`.    
Once the cell are executed, note the different text colors in the logging cells.

In [1]:
info("Initializing parameters")

INFO: Initializing parameters


In [2]:
warn("Problem inside the data file")

In [3]:
error("Cannot read this ODV file")

LoadError: [91mCannot read this ODV file[39m

The call to `error` throws a *Stracktrace*, which is maybe not expected.
### Configuration
At some point we might not be interested in the `info` logs, but of course we don't want to comment them out if the code, so we modify the configuration of the logging. However in the notebook, this doesn't change anything, the info messages are still ussued.

In [4]:
logging(kind=:warn)
info("Should not appear")

INFO: Should not appear


We can use a different prefix:

In [5]:
info("Start processing file", prefix="divand:")

divand:Start processing file


This can be useful if we want to have a unique logging file where the messages are related to the julia file that called them.      
We could also attach a time instance to the message:

In [6]:
using Dates

In [7]:
info("Start", prefix=string(now()) * " divand: ")
warn("File not found", prefix=string(now()) * " divand: ")

2018-03-20T10:05:51.55 divand: Start
2018-03-20T10:05:51.617 divand: File not found


Interestingly, the `error` function does not accept the prefix, even if it appears similar to `info` and `warn`:

In [8]:
error("No bathymetry", prefix=string(now()) * " divand: ")

LoadError: [91mfunction error does not accept keyword arguments[39m

In [9]:
?logging

search: logging



```
logging(io [, m [, f]][; kind=:all])
logging([; kind=:all])
```

Stream output of informational, warning, and/or error messages to `io`, overriding what was otherwise specified.  Optionally, divert stream only for module `m`, or specifically function `f` within `m`.  `kind` can be `:all` (the default), `:info`, `:warn`, or `:error`.  See `Base.log_{info,warn,error}_to` for the current set of redirections.  Call `logging` with no arguments (or just the `kind`) to reset everything.


### Writing into a file
It is straightformard to redirect the output into a file, in which the format (colors, bold typing) is converved.     

In [10]:
ff = open("./divand.log", "w")
info(ff, "Check 1, 2")
warn(ff, "We've got a situation here")
close(ff)

In bash, use the `more` command to view the file content with the formatting.

In [11]:
run(`more ./divand.log`)

::::::::::::::
./divand.log
::::::::::::::
INFO: Check 1, 2


## Logging package
Now we use the external package and see that the method definitions are overwritten for `warn` and `info`.

In [12]:
using Logging

In [13]:
info("Starting interpolation")

With info, no message is displayed because we have to configure the logging level.

In [14]:
Logging.configure(level=DEBUG)
debug("Checking")
info("Starting")
warn("Strange stuff")
err("Problem")
critical("Crashhhhh")

20-mars 10:06:03:DEBUG:root:Checking
20-mars 10:06:03:INFO:root:Starting
20-mars 10:06:03:WARNING:root:Strange stuff
20-mars 10:06:03:ERROR:root:Problem
20-mars 10:06:03:CRITICAL:root:Crashhhhh


In [15]:
Logging.configure(level=ERROR)
debug("Checking")
info("Starting")
warn("Strange stuff")
err("Problem")
critical("Crashhhhh")

20-mars 10:06:05:ERROR:root:Problem
20-mars 10:06:05:CRITICAL:root:Crashhhhh


### Write to a file

In [16]:
Logging.configure(output=open("divand2.log", "a"))

Logger(root,ERROR,Union{IO, Logging.SysLog}[IOStream(<file divand2.log>)],root)

In [17]:
debug("Checking")
info("Starting")
warn("Strange stuff")
err("Problem")
critical("Crashhhhh")

The file content is visualised with `more` or other tools:

In [18]:
run(`more divand2.log`)

::::::::::::::
divand2.log
::::::::::::::
19-mars 22:23:34:ERROR:root:Problem
19-mars 22:23:34:CRITICAL:root:Crashhhhh


In [ ]:
We can also have different loggers for different parts of the code, but that's not necessary now.
# Conclusion
The external `Logging` seems a more convenient solution at the time of this notebook.